In [9]:
import os

import panel as pn
pn.extension()

import requests
import pandas as pd
import datetime as dt
import alpaca_trade_api as tradeapi
import hvplot.pandas
from MCForecastTools import MCSimulation

In [3]:
# Load .env environment variables
from dotenv import load_dotenv
load_dotenv()

True

In [4]:
def get_workday(date):
    date = pd.to_datetime(date)
    offset = max(1, (date.weekday() + 6) % 7 - 3)
    timedelta = dt.timedelta(offset)
    return pd.Timestamp((date - timedelta).date(),tz="America/New_York").isoformat()

In [5]:
# Set Alpaca API key and secret
alpaca_api_key = os.getenv("ALPACA_API_KEY")
alpaca_secret_key = os.getenv("ALPACA_SECRET_KEY")

api = tradeapi.REST(
    alpaca_api_key,
    alpaca_secret_key,
    api_version = "v2"
)

In [6]:
# Format current date as ISO format
end_date = get_workday("today")
two_yrs = dt.timedelta(days=(365*2))
start_date = get_workday((pd.to_datetime("today") - two_yrs).date())
# Set the tickers
tickers = ["AAPL", "MSFT", "AMZN", "FB", "GOOGL", "BRK.B", "JNJ", "V", "PG", "SPY"]
# Set timeframe to '1D' for Alpaca API
timeframe = "1D"
# Get current closing prices for SPY and AGG
# YOUR CODE HERE!
df_portfolio = api.get_barset(
    tickers,
    timeframe,
    start = start_date,
    end = end_date
).df

df_portfolio.head()

AAPL                                       AMZN  \
                             open    high     low   close    volume     open   
time                                                                           
2018-10-31 00:00:00-04:00  216.88  220.45  216.62  218.75  31900983  1569.11   
2018-11-01 00:00:00-04:00  219.07  222.36  216.81  222.05  40261246  1623.53   
2018-11-02 00:00:00-04:00  209.55  213.89  205.43  206.14  82917876  1678.59   
2018-11-05 00:00:00-05:00  204.30  204.39  198.17  201.59  51670747  1657.57   
2018-11-06 00:00:00-05:00  201.92  204.72  201.69  203.78  27934956  1618.35   

                                                                 ...     SPY  \
                                high      low    close   volume  ...    open   
time                                                             ...           
2018-10-31 00:00:00-04:00  1623.9100  1565.09  1599.19  4602373  ...  270.65   
2018-11-01 00:00:00-04:00  1670.4465  1598.44  1665.86  7682121  ...  271.62   
2018-11-02 00:00:00-04:00  1697.4400  1651.83  1665.58  6221611  ...  274.74   
2018-11-05 00:00:00-05:00  1658.0900  1596.36  1627.45  5231856  ...  272.44   
2018-11-06 00:00:00-05:00  1665.0000  1614.55  1642.86  4000487  ...  273.32   

                                                                  V            \
                             high     low   close    volume    open      high   
time                                                                            
2018-10-31 00:00:00-04:00  273.23  270.12  270.69  97427146  134.99  139.1200   
2018-11-01 00:00:00-04:00  273.73  270.38  273.51  72100016  139.00  140.8700   
2018-11-02 00:00:00-04:00  275.23  269.59  271.80  93545041  141.11  141.4546   
2018-11-05 00:00:00-05:00  274.01  271.35  273.38  55157537  139.80  140.2100   
2018-11-06 00:00:00-05:00  275.30  273.25  275.12  49731823  139.80  140.9100   

                                                    
                              low   close   volume  
time                                                
2018-10-31 00:00:00-04:00  134.99  137.84  8880405  
2018-11-01 00:00:00-04:00  137.49  140.82  7601032  
2018-11-02 00:00:00-04:00  137.85  139.70  7017962  
2018-11-05 00:00:00-05:00  138.07  139.79  5472273  
2018-11-06 00:00:00-05:00  139.72  140.78  4203592  

[5 rows x 50 columns]

In [7]:
def risk_chart ():
    df = pd.DataFrame()
    for symbol in tickers:
        df[symbol] = df_portfolio[symbol]["close"]
    df_pct = df.pct_change().dropna()
    df_pct.index = df_pct.index.date
    df_pct = df_pct.tail(120)
    limit = (min(df_pct.min())-0.01, max(df_pct.max())+0.01)
    box_chart = df_pct.hvplot.box(
        ylim = limit,
        figsize = (10,20),
        title = "6 Months Percent Return Range Risk",
        xlabel = "Stock Symbol",
        ylabel = "Percent Returns"
    )
    return box_chart

In [8]:
risk_chart()

:BoxWhisker   [Variable]   (value)